# Dynamic Configuration and Chaining in Mirascope

This notebook provides a detailed introduction to using Dynamic Configuration and Chaining in Mirascope. We'll cover various examples ranging from basic usage to more complex chaining techniques.

For more information on these topics, refer to the following documentation:
- [Dynamic Configuration](https://docs.mirascope.io/learn/dynamic_configuration/)
- [Chaining](https://docs.mirascope.io/learn/chaining/)

1. [Setup](#Setup)
2. [Dynamic Configuration](#Dynamic-Configuration)
   - [Basic Usage](#Basic-Usage)
   - [Computed Fields](#Computed-Fields)
   - [Dynamic Tools](#Dynamic-Tools)
3. [Chaining](#Chaining)
   - [Function-based Chaining](#Function-based-Chaining)
   - [Chaining with Computed Fields](#Chaining-with-Computed-Fields)
4. [Advanced Techniques](#Advanced-Techniques)
   - [Combining Dynamic Config and Chaining](#Combining-Dynamic-Config-and-Chaining)
   - [Error Handling in Chains](#Error-Handling-in-Chains)
5. [Conclusion](#Conclusion)

## Setup

First, let's install Mirascope and set up our environment. We'll use OpenAI for our examples, but you can adapt these to other providers supported by Mirascope. For more information on supported providers, see the [Calls documentation](https://docs.mirascope.io/learn/calls/).

In [ ]:
!pip install "mirascope[openai]"

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-api-key-here"

## Dynamic Configuration

Dynamic Configuration in Mirascope allows you to modify the behavior of LLM calls at runtime based on input arguments or other conditions. For more details, see the [Dynamic Configuration documentation](https://docs.mirascope.io/learn/dynamic_configuration/).

### Basic Usage

In [2]:
from mirascope.core import BaseDynamicConfig, Messages, openai, prompt_template


@openai.call("gpt-4o-mini")
@prompt_template("Recommend a {genre} book")
def recommend_book(genre: str, creativity: float) -> BaseDynamicConfig:
    return {
        "messages": [Messages.User(f"Recommend a {genre} book")],
        "call_params": {"temperature": creativity},
    }


# Low creativity recommendation
response = recommend_book("mystery", 0.2)
print("Low creativity:", response.content)

# High creativity recommendation
response = recommend_book("mystery", 0.8)
print("High creativity:", response.content)

Low creativity: I recommend "The Guest List" by Lucy Foley. This gripping psychological thriller is set on a remote Irish island where a glamorous wedding takes a dark turn when a murder occurs. The story is told from multiple perspectives, revealing secrets and lies among the guests. With its intricate plot and well-drawn characters, it's a captivating read for any mystery lover. Enjoy!
High creativity: One highly recommended mystery book is **"The Girl with the Dragon Tattoo"** by Stieg Larsson. This gripping novel follows journalist Mikael Blomkvist as he investigates the decades-old disappearance of a wealthy industrialist's niece, with the help of the enigmatic hacker Lisbeth Salander. The story weaves together themes of family secrets, corruption, and personal redemption, making it a compelling read for mystery enthusiasts. If you're looking for something more recent, consider **"The Last House on Needless Street"** by Catriona Ward—a psychological thriller with plenty of twists 

### Computed Fields

When using the `Messages.Type` return for writing prompt templates, you can inject computed fields directly into the formatted strings:

In [3]:
@openai.call("gpt-4o-mini")
@prompt_template()
def recommend_book_by_age(genre: str, age: int) -> Messages.Type:
    reading_level = "adult"
    if age < 12:
        reading_level = "elementary"
    elif age < 18:
        reading_level = "young adult"
    return f"Recommend a {genre} book with a reading level of {reading_level}"


response = recommend_book_by_age("fantasy", 15)
print(response.content)

I recommend **"Six of Crows" by Leigh Bardugo**. This fantasy novel features a diverse cast of characters and is set in a richly built world full of intricate politics, magic, and heists. The story follows a group of outcasts and misfits as they plan a dangerous heist, and it combines action, adventure, and character-driven storytelling. It's highly regarded in the young adult fantasy genre and is the first book in a duology, making it a great choice for fans of fantasy and thrilling plots.


When using string templates, you can use computed fields to dynamically generate or modify template variables used in your prompt. For more information on prompt templates, see the [Prompts documentation](https://docs.mirascope.io/learn/prompts/).

In [4]:
@openai.call("gpt-4o-mini")
@prompt_template("Recommend a {genre} book with a reading level of {reading_level}")
def recommend_book_by_age(genre: str, age: int) -> openai.OpenAIDynamicConfig:
    reading_level = "adult"
    if age < 12:
        reading_level = "elementary"
    elif age < 18:
        reading_level = "young adult"
    return {"computed_fields": {"reading_level": reading_level}}


response = recommend_book_by_age("fantasy", 15)
print(response.content)

I recommend **"Throne of Glass" by Sarah J. Maas**. It's a captivating young adult fantasy novel featuring a strong female protagonist, Celaena Sardothien, who is an assassin in a corrupt kingdom. The story is filled with intrigue, magic, and romance, making it a great choice for young adult readers who enjoy adventurous and character-driven tales. The book is the first in a series, so if you enjoy it, there's plenty more to explore!


### Dynamic Tools

You can dynamically configure which tools are available to the LLM based on runtime conditions. Here's a simple example using a basic tool function:

In [5]:
def format_book(title: str, author: str, genre: str) -> str:
    """Format a book recommendation."""
    return f"{title} by {author} ({genre})"


@openai.call("gpt-4o-mini")
@prompt_template()
def recommend_book_with_tool(genre: str) -> openai.OpenAIDynamicConfig:
    return {
        "messages": [Messages.User(f"Recommend a {genre} book")],
        "tools": [format_book],
    }


response = recommend_book_with_tool("mystery")
if response.tool:
    print(response.tool.call())
else:
    print(response.content)

The Girl with the Dragon Tattoo by Stieg Larsson (Mystery)


For more advanced usage of tools, including the `BaseToolKit` class, please refer to the [Tools documentation](https://docs.mirascope.io/latest/learn/tools/) and the [Tools and Agents Getting Started Guide](https://docs.mirascope.io/latest/cookbook/agents/).

## Chaining

Chaining in Mirascope allows you to combine multiple LLM calls or operations in a sequence to solve complex tasks. Let's explore two main approaches to chaining: function-based chaining and chaining with computed fields.

### Function-based Chaining

In function-based chaining, you call multiple functions in sequence, passing the output of one function as input to the next. This approach requires you to manage the sequence of calls manually.

In [6]:
@openai.call("gpt-4o-mini")
@prompt_template()
def summarize(text: str) -> str:
    return f"Summarize this text: {text}"


@openai.call("gpt-4o-mini")
@prompt_template()
def translate(text: str, language: str) -> str:
    return f"Translate this text to {language}: {text}"


original_text = "Long English text here..."
summary = summarize(original_text)
translation = translate(summary.content, "french")
print(translation.content)

Bien sûr ! Veuillez fournir le texte que vous aimeriez que je résume.


### Nested Chaining

You can easily create a single function that calls the entire chain simply by calling each part of the chain in the function body of the corresponding parent:

In [7]:
@openai.call("gpt-4o-mini")
@prompt_template()
def summarize(text: str) -> str:
    return f"Summarize this text: {text}"


@openai.call("gpt-4o-mini")
@prompt_template()
def summarize_and_translate(text: str, language: str) -> str:
    summary = summarize(original_text)
    return f"Translate this text to {language}: {summary.content}"


original_text = "Long English text here..."
translation = summarize_and_translate(original_text, "french")
print(translation.content)

Bien sûr ! Veuillez fournir le texte que vous souhaitez que je résume.


### Chaining with Computed Fields

Chaining with computed fields allows you to better trace your nested chains since the full chain of operations will exist in the response of the single function (rather than having to call and track each part of the chain separately).

In [9]:
@openai.call("gpt-4o-mini")
@prompt_template()
def summarize(text: str) -> str:
    return f"Summarize this text: {text}"


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {summary}")
def summarize_and_translate(text: str, language: str) -> BaseDynamicConfig:
    return {"computed_fields": {"summary": summarize(text)}}


response = summarize_and_translate("Long English text here...", "french")
print("Translation:", response.content)
print(
    "\nComputed fields (including summary):", response.dynamic_config["computed_fields"]
)

Translation: Veuillez fournir le texte que vous aimeriez que je résume.

Computed fields (including summary): {'summary': OpenAICallResponse(metadata={}, response=ChatCompletion(id='chatcmpl-AAkXulav2CQsWjfwmDoNcsK0BKSVB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Please provide the text you would like me to summarize.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1727125566, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_1bb46167f9', usage=CompletionUsage(completion_tokens=11, prompt_tokens=18, total_tokens=29, completion_tokens_details={'reasoning_tokens': 0})), tool_types=None, prompt_template=None, fn_args={'text': 'Long English text here...'}, dynamic_config={'messages': [BaseMessageParam(role='user', content='Summarize this text: Long English text here...')]}, messages=[{'role': 'user', 'content': 'Summarize this text: Long English te

As you can see, with computed fields, you get access to both the final translation and the intermediate summary in a single response. This approach provides better traceability and can be particularly useful for debugging and understanding the entire chain of operations without the need to manage multiple separate function calls.

Of course, you can always put the computed fields in the dynamic configuration without using string templates for the same effect:

In [10]:
@openai.call("gpt-4o-mini")
@prompt_template()
def summarize(text: str) -> str:
    return f"Summarize this text: {text}"


@openai.call("gpt-4o-mini")
@prompt_template("Translate this text to {language}: {summary}")
def summarize_and_translate(text: str, language: str) -> BaseDynamicConfig:
    summary = summarize(text)
    return {
        "messages": [
            Messages.User(f"Translate this text to {language}: {summary.content}"),
        ],
        "computed_fields": {"summary": summary},
    }


response = summarize_and_translate("Long English text here...", "french")
print("Translation:", response.content)
print(
    "\nComputed fields (including summary):", response.dynamic_config["computed_fields"]
)

Translation: Bien sûr ! Veuillez fournir le texte que vous aimeriez que je résume.

Computed fields (including summary): {'summary': OpenAICallResponse(metadata={}, response=ChatCompletion(id='chatcmpl-AAkYkYoxPqIaDWqaM5MLCUvlGlGUz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Sure! Please provide the text that you would like me to summarize.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1727125618, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_1bb46167f9', usage=CompletionUsage(completion_tokens=14, prompt_tokens=18, total_tokens=32, completion_tokens_details={'reasoning_tokens': 0})), tool_types=None, prompt_template=None, fn_args={'text': 'Long English text here...'}, dynamic_config={'messages': [BaseMessageParam(role='user', content='Summarize this text: Long English text here...')]}, messages=[{'role': 'user', 'content': 'Summarize this

### Error Handling in Chains

Implementing robust error handling is crucial in complex chains:

In [12]:
from openai import OpenAIError


@openai.call("gpt-4o-mini")
@prompt_template()
def summarize(text: str) -> str:
    return f"Summarize this text: {text}"


@openai.call("gpt-4o-mini")
@prompt_template()
def translate(text: str, language: str) -> str:
    return f"Translate this text to {language}: {text}"


def process_text_with_error_handling(text: str, target_language: str):
    try:
        summary = summarize(text).content
    except OpenAIError as e:
        print(f"Error during summarization: {e}")
        summary = text  # Fallback to original text if summarization fails

    try:
        translation = translate(summary, target_language).content
        return translation
    except OpenAIError as e:
        print(f"Error during translation: {e}")
        return summary  # Fallback to summary if translation fails


result = process_text_with_error_handling("Long text here...", "French")
print("Processed Result:", result)

Processed Result: Bien sûr ! Veuillez fournir le texte que vous souhaitez résumer, et je serai heureux de vous aider.


## Conclusion

This notebook has demonstrated various techniques for using Dynamic Configuration and Chaining in Mirascope. These powerful features allow you to create flexible, efficient, and complex LLM-powered applications. By combining these techniques, you can build sophisticated AI systems that can adapt to different inputs and requirements while maintaining robustness and traceability.

Remember to always consider error handling, especially in complex chains, to ensure your applications are resilient to potential issues that may arise during LLM calls or processing steps.